In [1]:
import pandas as pd
import pprint
import matplotlib.pyplot as plt
import numpy as np
import folium
from geopy.geocoders import Nominatim
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
bridge_csv = pd.read_csv("C:/Users/samka/Downloads/NTAD_National_Bridge_Inventory_-6282134062105639862.csv", low_memory=False)
philly_bridges = bridge_csv[(bridge_csv["STATE_CODE_001"] == 42) & (bridge_csv["COUNTY_CODE_003"] == 101) 
                            & (bridge_csv["MAINTENANCE_021"] == 4)]
philly_bridges

,OBJECTID,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,...,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA,STATUS,DATE,LATDD,LONGDD,x,y
452996,452997,42,000000000007193,1,3,1,00000,0,06,101,...,42,G,7,237.90,AM,1202,40.103450,-74.961092,-74.961092,40.103450
466573,466574,42,000000000038954,1,3,1,00000,0,06,101,...,42,F,6,631.68,AM,1202,40.017092,-75.211428,-75.211428,40.017092
466579,466580,42,000000000038968,1,3,1,00000,0,06,101,...,42,P,4,2688.70,ST,723,39.963467,-75.186778,-75.186778,39.963467
466580,466581,42,000000000038970,1,3,1,00000,0,06,101,...,42,P,4,439.02,ST,723,39.963861,-75.185025,-75.185025,39.963861
466616,466617,42,000000000039034,1,3,1,00000,0,06,101,...,42,F,5,2384.64,ST,723,40.035356,-75.208878,-75.208878,40.035356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471579,471580,42,000000000054173,1,5,0,00000,3,06,101,...,42,G,7,592.02,ST,723,39.958900,-75.170372,-75.170372,39.958900
471995,471996,42,000000000055933,1,5,0,00000,3,06,101,...,42,G,7,795.60,ST,723,39.959400,-75.173906,-75.173906,39.959400
471997,471998,42,000000000055937,1,5,0,00000,1,06,101,...,42,G,7,838.95,ST,723,39.959500,-75.175500,-75.175500,39.959500
472286,472287,42,000000000057140,1,5,0,00000,2,06,101,...,42,P,3,82.08,ST,723,39.960194,-75.159694,-75.159694,39.960194


In [ ]:
philly_bridges = philly_bridges[['OBJECTID', 'LATDD', 'LONGDD', 'YEAR_BUILT_027', 'BRIDGE_CONDITION',
                                 'ADT_029', 'DECK_WIDTH_MT_052', 'STRUCTURE_LEN_MT_049' ]]
philly_bridges

In [ ]:
philly_bridges.rename(columns={"OBJECTID": "Bridge_ID",
                          "LATDD": "Latitude",
                          "LONGDD": "Longitude",
                          "YEAR_BUILT_027": "Year_Built",
                          "BRIDGE_CONDITION": "Bridge_Condition",
                          "ADT_029": "Average Daily_Traffic",
                          "DECK_WIDTH_MT_052": "Deck_Width",
                          "STRUCTURE_LEN_MT_049": "Structure_Length"}, inplace=True)

philly_bridges

In [ ]:
philly_bridges.loc[:, 'SqFt'] = philly_bridges['Deck_Width'] * philly_bridges['Structure_Length']
philly_bridges

In [ ]:
philly_bridges.loc[:,"Repair_Cost_Est"] = philly_bridges["SqFt"] * 314
philly_bridges

In [ ]:
philly_bridges.loc[:,"Replacement_Cost_Est"] = philly_bridges["SqFt"] * 461.76
philly_bridges

In [ ]:
total_repair_cost_est = philly_bridges["Repair_Cost_Est"].sum()
total_replacement_cost_est = philly_bridges["Replacement_Cost_Est"].sum()
difference = total_replacement_cost_est - total_repair_cost_est
def format_currency(amount):
    return '${:,.2f}'.format(amount)
total_repair_cost_est_formatted = format_currency(total_repair_cost_est)
total_replacement_cost_est_formatted = format_currency(total_replacement_cost_est)
difference_formatted = format_currency(difference)
print('Total Repair Cost Est:', total_repair_cost_est_formatted)
print('Total Replacement Cost Est:', total_replacement_cost_est_formatted)
print('Total Savings By Repairing Versus Replacing:', difference_formatted)

In [ ]:
labels = ['Total Repair Cost', 'Total Replacement Cost', 'Cost Difference']
values = [total_repair_cost_est, total_replacement_cost_est, difference]

x_pos = np.arange(len(labels))

plt.figure(figsize=(10,6))
bars = plt.bar(labels, values, color=['blue', 'green', 'orange'], width=0.25)
plt.title('Repair Costs versus Replacement Costs')
plt.ylabel('Costin Billions ($)')
for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), format_currency(value),
             ha='center', va='bottom', color='black', fontsize=10)
plt.xticks(x_pos, labels, rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
def format_currency(amount):
    return '${:,.2f}'.format(amount)
philly_bridges['Repair_Cost_Est'] = philly_bridges['Repair_Cost_Est'].apply(format_currency)
philly_bridges['Replacement_Cost_Est'] = philly_bridges['Replacement_Cost_Est'].apply(format_currency)
philly_bridges

In [ ]:
philly_bridges["Bridge_Condition"] = philly_bridges["Bridge_Condition"].replace({'P': 'Poor', 'F': 'Fair', 'G': 'Good'})

philly_bridges


In [ ]:
geolocator = Nominatim(user_agent="bridge_address")

In [ ]:
def reverse_geocode(row):
    location = geolocator.reverse((row["Latitude"], row['Longitude']), exactly_one=True)
    if location:
        return location.address
    else:
        return "N/A"
    
#Add in Address column with reverse geocode data
philly_bridges["Address"] =philly_bridges.apply(reverse_geocode, axis=1)

philly_bridges

In [ ]:
words_to_delete = [', Philadelphia', ', Philadelphia County', ', Bucks', ' County', 
                   ', Bensalem Township', ', Pennsylvania', ', United States']

def remove_words(text, words):
    pattern = r'\b(?:{})\b'.format('|'.join(words))
    return re.sub(pattern, '', text)

philly_bridges['Address'] = philly_bridges['Address'].apply(lambda x: remove_words(x, words_to_delete))

philly_bridges

In [ ]:
philly_bridges.dtypes

In [ ]:
philly_bridges.to_csv('Philly_Bridge.csv', index=False)